#### 원하는 데이터 크롤링
- 해당 데이터가 문서화 되어야 데이터 활용
- urllib -> 파이썬 내장 라이브러리 (설치 할 필요X)
- openpyxl 라이브러리
- pip install openpyxl

- (기존 -> 주피터 노트북으로 데이터 찾아서(*for문) 즉석에서 값을 보고 끝)
1. 데이터 찾기
2. 찾아온 데이터 데이터 양식에 넣고
3. 엑셀에다가 전달

In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# openpyxl 라이브러리 찾아오기
import openpyxl

# Workbook() 프로토타입 객체 실행시켜서 하나의 엑셀파일을 생성하겠다는 의미
excel_file = openpyxl.Workbook()

#엑셀파일 내부에는 기본적으로 Sheet가 1개 존재 => 해당 시트를 선택(*active)
excel_sheet = excel_file.active

# 선택한 엑실시트 내부에 데이터 추가 및 삽입하기
excel_sheet.append(["data1", "data2", "data3"])

# 엑셀파일 저장하기
excel_file.save(tmp.xlsx)

# 엑셀파일 종료하기
excel_file.close()

In [25]:
# 1. 데이터 크롤링 파트

import requests
from bs4 import BeautifulSoup

url = "https://davelee-fun.github.io/page2/"
product_lists = list()    #전역변수 : 모든 페이지 상품 데이터를 여기에 누적 저장

for page_num in range(10) :    # 0~9까지 총 10페이지 크롤링 반복
    if page_num == 0 :
        res = requests.get(url)    # 첫 페이지는 기본 URL 호출
    else :
        res = requests.get(f"{url}page{page_num+1}")    # 2페이지 부터 URL에 page번호 (0이 아닌 1부터 값이 나와서)
    soup = BeautifulSoup(res.content, "html.parser")    # 파싱 준비
    datas = soup.select("div.card")    # 한 페이지 내 모든 상품 카드 선택

    for data in datas : 
        product_name = data.select_one("div.card-body > h4.card-text")    # 지역변수 : 각 상품 카드 내 제목 선택
        product_data = data.select_one("div.wrapfooter span.post-date")    # 각 상품 카드 내 등록일 정보 선택 / 띄어쓰기(후손의미)
        product_info = [product_name.get_text().strip(), product_data.get_text().strip()]
        product_lists.append(product_info)    # 전역변수 product_lists에 지역변수 product_info추가


In [29]:
# 2. 엑셀 시트 저장용 함수 생성 파트

import openpyxl    # 엑셀 작업을 위한 외부 라이브러리

def write_excel_template(filename, sheetname, listdata) :    # 엑셀 파일을 생성해서 리스트 데이터를 저장하는 함수 정의
    excel_file = openpyxl.Workbook()    # 새 엑셀 워크북 생성 (기본적으로 1개의 시트 포함)
    excel_sheet = excel_file.active    # 기본 활성화된 시트 선택
    excel_sheet.column_dimensions["A"].width = 100     # 첫번째 열(A열) 너비 100 설정
  
    if sheetname != "" :    # 시트명 비어있지 않은 경우 시트 이름 변경
        excel_sheet.title = sheetname   

    for item in listdata :    # 리스트 데이터를 한 줄 씩 반복해서 엑셀에 한 행씩 추가
        excel_sheet.append(item)    # 각 item은 [상품명, 등록일] 같은 형태 리스트
        
    excel_file.save(filename)    # 지정한 파일명으로 엑셀 파일 저장
    excel_file.close()    # 엑셀파일닫기   

In [30]:
# 3. 엑셀 시트 저장용 함수 호출 파트

# 크롤링 결과로 수집한 product_lists데이러 -> 상품정보라는 시트명으로 tep.xlsx 엑셀 파일에 저장
write_excel_template("tep.xlsx", "상품정보", product_lists)